### Get Azure Machine Learning Workspace

Workspace is used to retreive credentials and secrets in Key Vault to connect to Azure Data Explorer.

In [2]:
from aml_obs.drift import Drift_Analysis
from azureml.core import Workspace
from aml_obs.drift import launch_dashboard

ws = Workspace.from_config()
drift_analysis = Drift_Analysis(ws)

### Option 1: Local Dashboard with Dash

Run cell below and copy the link into browser to access the dashboard. Once in the dashboard, interact with the widgets to create visualizations from avaiable data. Use ```irisdata``` created in part 0 as a sample table if needed. 

In [4]:
launch_dashboard(drift_analysis)

Dash app running on http://127.0.0.1:8050/


### Option 2: Upload Dashboard to Azure Data Explorer

If a centralized dashboard is preferred, go to Azure Data Explorer Dashboards and import the quick_start/ADX_dashboards/databoard_drift_detection.json file to create a dashboard.

Review the instructions on how to import the dashboard [here.](https://docs.microsoft.com/en-us/azure/data-explorer/azure-data-explorer-dashboards#to-create-new-dashboard-from-a-file)

Default parameters may need to updated to ensure proper connection.